In [20]:
import torch
import requests
from tabpfn import TabPFNClassifier
import os
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.data import random_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import random
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [2]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, noise, transform=None, target_transform=None, drop=None, target=None):
        self.dataframe = dataframe
        if drop != None:
            self.X = dataframe.drop(drop, axis=1).values
        else:
            self.X = dataframe.values
        
        self.y = dataframe[target].values
        self.transform = transform
        self.target_transform = target_transform
        self.noise = noise

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        item, label = self.X[idx], self.y[idx]
        return item, label

    def get_noise(self):
        return self.noise

In [3]:
Noise_0_dataframe = pd.read_csv("../Data/Assignment1/data_0_noise")
Noise_Low_dataframe = pd.read_csv("../Data/Assignment1/data_Low_noise")
Noise_High_dataframe = pd.read_csv("../Data/Assignment1/data_High_noise")

In [4]:
class_index = list(Noise_0_dataframe["era"].unique())
class_index_noise = list(Noise_Low_dataframe["era"].unique())
class_index_t10v_noise = list(Noise_Low_dataframe["target_10_val"].unique())

def encode(value, class_index = class_index):
    return class_index.index(value)

def encode_noise(value, class_index = class_index_noise):
    return class_index.index(value)

def encode_noise_t10v(value, class_index = class_index_t10v_noise):
    return class_index.index(value)


Noise_0_dataframe["era"] = Noise_0_dataframe["era"].apply(encode)
Noise_Low_dataframe["era"] = Noise_Low_dataframe["era"].apply(encode_noise)
Noise_High_dataframe["era"] = Noise_High_dataframe["era"].apply(encode_noise)
Noise_Low_dataframe["target_10_val"] = Noise_Low_dataframe["target_10_val"].apply(encode_noise_t10v)
Noise_High_dataframe["target_10_val"] = Noise_High_dataframe["target_10_val"].apply(encode_noise_t10v)

In [5]:
Noise_0_dataset_era = CustomDataset(Noise_0_dataframe, "0",drop = ["row_num","day","era","target_10_val","target_5_val"], target = "era")
Noise_Low_dataset_era = CustomDataset(Noise_Low_dataframe, "Low", drop = ["row_num","day","era","target_10_val","target_5_val","data_type"], target = "era")
Noise_High_dataset_era = CustomDataset(Noise_High_dataframe, "High", drop = ["row_num","day","era","target_10_val","target_5_val","data_type"], target = "era")
Noise_Low_dataset_t10v = CustomDataset(Noise_Low_dataframe, "Low", drop = ["row_num","day","era","target_10_val","target_5_val","data_type"], target = "target_10_val")
Noise_High_dataset_t10v = CustomDataset(Noise_High_dataframe, "High", drop = ["row_num","day","era","target_10_val","target_5_val","data_type"], target = "target_10_val")
Noise_0_train_era, Noise_0_test_era = random_split(Noise_0_dataset_era, [int(len(Noise_0_dataset_era)*0.8), int(len(Noise_0_dataset_era)*0.2)])
Noise_Low_train_era, Noise_Low_test_era = random_split(Noise_Low_dataset_era, [int(len(Noise_Low_dataset_era)*0.8), int(len(Noise_Low_dataset_era)*0.2)])
Noise_High_train_era, Noise_High_test_era = random_split(Noise_High_dataset_era, [int(len(Noise_High_dataset_era)*0.8), int(len(Noise_High_dataset_era)*0.2)])
Noise_Low_train_t10v, Noise_Low_test_t10v = random_split(Noise_Low_dataset_t10v, [int(len(Noise_Low_dataset_t10v)*0.8), int(len(Noise_Low_dataset_t10v)*0.2)])
Noise_High_train_t10v, Noise_High_test_t10v = random_split(Noise_High_dataset_t10v, [int(len(Noise_High_dataset_t10v)*0.8), int(len(Noise_High_dataset_t10v)*0.2)])

In [10]:
#Setting up dataloaders
Noise_0_era_train_loader = DataLoader(Noise_0_train_era, batch_size=512, shuffle=True)
Noise_0_era_test_loader = DataLoader(Noise_0_test_era, batch_size=512, shuffle=True)
########################################################################################
Noise_Low_era_train_loader = DataLoader(Noise_Low_train_era, batch_size=512, shuffle=True)
Noise_Low_era_test_loader = DataLoader(Noise_Low_test_era, batch_size=512, shuffle=True)
########################################################################################
Noise_High_era_train_loader = DataLoader(Noise_High_train_era, batch_size=512, shuffle=True)
Noise_High_era_test_loader = DataLoader(Noise_High_test_era, batch_size=512, shuffle=True)
########################################################################################
Noise_Low_t10v_train_loader = DataLoader(Noise_Low_train_t10v, batch_size=1000, shuffle=True)
Noise_Low_t10v_test_loader = DataLoader(Noise_Low_test_t10v, batch_size=512, shuffle=True)
########################################################################################
Noise_High_t10v_train_loader = DataLoader(Noise_High_train_t10v, batch_size=1000, shuffle=True)
Noise_High_t10v_test_loader = DataLoader(Noise_High_test_t10v, batch_size=512, shuffle=True)
########################################################################################


In [29]:
def predicition_assembler(predictions,probabilities):
    final_winner=[]
    no_of_models=len(predictions)
    batch_size=len(predictions[0])
    for i in range(batch_size):
        winner_dict={}
        for j in range(no_of_models):
            if predictions[j][i] not in winner_dict.keys():
                winner_dict[predictions[j][i]]=0
            winner_dict[predictions[j][i]]+=probabilities[j][i]
        final_winner.append(max(winner_dict,key=winner_dict.get))
    return final_winner

def fit_test_tabpfn(train_dataloader,test_dataloader,no_of_models_to_ensemble=1,ensemble_config=1):
    all_tabpfns=[]
    for data,target in tqdm(train_dataloader, desc="FITTING"):
        classifier = TabPFNClassifier(device=device, N_ensemble_configurations=ensemble_config)
        classifier.fit(data,target)
        all_tabpfns.append(classifier)

    total=0
    correct=0
    for data,target in tqdm(test_dataloader, desc="TESTING"):
        each_model_prob=[]
        each_model_pred=[]
        random_models = random.sample(all_tabpfns, no_of_models_to_ensemble)
        for model in random_models:
            y_pred,p_pred=model.predict(data,return_winning_probability=True)
            each_model_prob.append(p_pred)
            each_model_pred.append(y_pred)
        y_pred_summ=predicition_assembler(each_model_pred,each_model_prob)
        total+=len(y_pred_summ)
        correct+=sum(1 for p, t in zip(y_pred_summ, target) if p == t)
    print(f"Accuracy: {correct/total:.4f}") 

In [30]:
fit_test_tabpfn(train_dataloader=Noise_Low_t10v_train_loader,test_dataloader=Noise_Low_t10v_test_loader,no_of_models_to_ensemble=10,ensemble_config=4)

TESTING: 100%|██████████| 122/122 [13:20<00:00,  6.56s/it]

Accuracy: 0.7792


In [31]:
fit_test_tabpfn(train_dataloader=Noise_High_t10v_train_loader,test_dataloader=Noise_High_t10v_test_loader,no_of_models_to_ensemble=10,ensemble_config=2)

TESTING: 100%|██████████| 98/98 [07:51<00:00,  4.81s/it]

Accuracy: 0.6119
